# Estruturas Criptográficas - Criptografia e Segurança da Informação

[Grupo 03](https://paper.dropbox.com/doc/Estruturas-Criptograficas-2023-2024-Trabalhos-Praticos-8WcsdZARGLv0nXS9KasmK)

(PG54177) Ricardo Alves Oliveira 

(PG54236) Simão Oliveira Alvim Barroso

## TP3 - Exercício 3

3. O algoritmo de Boneh e Franklin (BF) discutido no +Capítulo 5b:  Curvas Elípticas e sua Aritmética é uma tecnica fundamental na chamada “Criptografia Orientada à Identidade”. Seguindo as orientações definidas nesse texto, pretende-se construir usando Sagemath uma classe Python que implemente este criptosistema.

## Resolução

Para resolver este exercício começamos por importar a biblioteca `sagemath`.

In [160]:
from sage.all import *

q = None
p = None
f = None
G = None
z = None
s = None
beta = None
E2 = None
nonce = 54345264

Um emparelhamento  tate  definido num grupo de torção $$\mathbb{G}$$ de ordem prima $$q$$ como vimos na secção de emparelhamentos.  O grupo $$\mathbb{G}\,$$ tem o gerador $$G$$.

Emparelhamento de tate.

De seguida definimos as funções auxiliares do criptosistema com recurso ao sagemath seguindo as orientações abaixo:

$$\mathsf{Zr}\;\colon\;\mathbb{N}\,\to\,\mathbb{Z}_q$$ 
$$f\;\colon\;\mathbb{F}_{p^2}\,\to\,\mathbb{Z}$$
$$h \;\colon\; \mathsf{Bytes} \,\to\, \mathbb{Z}$$
$$H\,\colon\,\mathbb{Z}\,\to\,\mathbb{Z}_q$$
$$g\;\colon\;\mathbb{Z}\,\to\,\mathbb{G}\quad$$ 
$$\mathsf{ID}\;\colon\;\mathsf{Bytes}\,\to\,\mathbb{G}\quad$$


In [161]:
# Gerador pseudo-aleatório
def Zr():
    set_random_seed(nonce)
    random_number = randint(0, q - 1)
    return random_number

# Função de hash f
def f_hash(data):
    return hash(data)

# Função de hash h
def h(bytes_seq):
    return hash(bytes_seq)

# Função de hash H
def H(integer):
    return hash(integer)%q

# Função g
def g(n):
    return n * G

# Função ID
def ID(m):
    return g(h(m))

def phi(P):         
    (x,y) = P.xy()
    return E2(z*x,y)

def TateX(P,Q,l=1):      
    return P.tate_pairing(phi(Q), q, 2)^l


In [162]:
def KeyGen(lbd):
    global q, p, f, G, z, s, beta, E2
    lbd = 8
    bq = 2^(lbd-1)
    bp = 2^lbd-1
    q = random_prime(bp,lbound=bq)
    
    t = q*3*2^(bp - bq)
    while not is_prime(t-1):
        t = t << 1
    
    p = t - 1
    
    
    Fp     = GF(p)         
    R.<z>  = Fp[]          
    f     = R(z^2 + z + 1)
    Fp2.<z> = GF(p^2, modulus=f)   
    
    
    E2 = EllipticCurve(Fp2, [0,1])
    
    cofac = (p + 1)//q
    G = cofac * E2.random_point()
    
    s = Zr()
    beta = g(s)

    return (s, beta)


In [163]:

print("KeyGen(8):", KeyGen(8))
print("Zr:", Zr())
print("f(120):", f_hash(120))
print("h(b'120'):", h(b'120'))
print("H(1209):", H(1209))
print("g(7879804275796629580):", g(7879804275796629580))
print("ID(b'120'):", ID(b'120'))

KeyGen(8): (43, (2273509137437079069073044363616197667795552*z + 2586539837687654800395197499527599141963365 : 1204686391688177408324022811118764308618635*z + 138873665520041564022499166115982588562199 : 1))
Zr: 43
f(120): 120
h(b'120'): 6309581710264563278
H(1209): 44
g(7879804275796629580): (447862481297099378476202810527519997145296*z + 1461048300905744039827616491204872797300801 : 555054075434616202784666178564378042288473*z + 2992393674203829586482044603163617514151907 : 1)
ID(b'120'): (301152104990636760498829661831703504363451*z + 3170673939305747100803916162932807622929809 : 1387567861736718772149878436344543592901995*z + 3706628427427890658021697706745543624310667 : 1)


$$key\;\gets\;(\,\vartheta \;d \gets \mathsf{ID}(\mathit{id})\;\centerdot\; s* d\,)$$

In [164]:
def KeyExtract(id):
    d = ID(id)
    key = s * d
    return key

key = KeyExtract(b'120')

print("KeyExtract(b'120'):",key)

KeyExtract(b'120'): (3362662182034269113809441857599421975414990*z + 3268573714501534688373970942333921246643851 : 2595618829414932721530002935789100233440100*z + 176330492917635750850022489055232096125347 : 1)



$$\mathsf{in\_ encrypt}(\mathit{id},x)\;\equiv$$ 
$$\vartheta\,d\gets\mathsf{ID}(\mathit{id})\,\centerdot\,\vartheta\,v\gets\mathsf{Zr}\,\centerdot\,\vartheta\,a\gets H(v\oplus x)\,\centerdot\,\vartheta\,\mu\gets\mathbf{ex}(\beta,d,a)\,\centerdot\,\langle x,v,a,\mu\rangle$$

In [165]:
def in_encrypt(id, x):
    d = ID(id)
    v = Zr()
    a = H(v ^^ x)
    mu = TateX(beta, d, a)
    return v, a, mu

v, a, mu = in_encrypt(b'120', 120)

print("in_encrypt(b'120', 120):", (v, a, mu))

in_encrypt(b'120', 120): (43, 83, 882560302452970277186011465024579528179184*z + 1477901369596924849905210313696855830796412)


$$\mathsf{out\_ encrypt}(x,v,a,\mu)\;\equiv$$
$$\vartheta\,\alpha\gets g(a)\,\centerdot\,\vartheta\,v'\gets v\oplus f(\mu)\,\centerdot\,x'\gets x\oplus H(v)\,\centerdot\,\langle \alpha,v',x'\rangle$$

In [166]:
def out_encrypt(x, v, a, mu):
    alpha = g(a)
    v_prime = v ^^ f_hash(mu)
    x_prime = x ^^ H(v)
    return (alpha, v_prime, x_prime)

print("out_encrypt(120,v,a,mu):", out_encrypt(120, v, a, mu))

out_encrypt(120,v,a,mu): ((847233263721477088871220453940206233687146*z + 2595127466026680811584227734898508292904506 : 1147434789891369584664262580891559908408026*z + 2753689016240793237272970083846994781883175 : 1), -3169791819069603670, 83)



$$\mathsf{Encrypt}(\mathit{id}, x) \;\equiv\;$$  
$$\vartheta \,x,v,a,\mu \gets \mathsf{in}(\mathit{id},x)\,\centerdot\,\mathsf{out}(x,v,a,\mu)$$

In [167]:
def Encrypt(id, x):
    v, a, mu = in_encrypt(id, x)
    alpha, v_prime, x_prime = out_encrypt(x, v, a, mu)
    return (alpha, v_prime, x_prime)

alpha, v_prime, x_prime = Encrypt(b'120', 120)

print("Encrypt(b'120', 120):", (alpha, v_prime, x_prime))

Encrypt(b'120', 120): ((847233263721477088871220453940206233687146*z + 2595127466026680811584227734898508292904506 : 1147434789891369584664262580891559908408026*z + 2753689016240793237272970083846994781883175 : 1), -3169791819069603670, 83)



$$\mathsf{in\_ decrypt}(\mathit{key},\alpha, v', x')\;\equiv$$
$$\vartheta\,\mu \gets \mathbf{ex}(\alpha,\mathit{key},1)\,\centerdot\,\vartheta\,v \gets v'\oplus f(\mu)\,\centerdot\,x\gets x'\oplus H(v)\,\centerdot\,\langle \alpha,v,x\rangle$$

In [168]:
def in_decrypt(key, alpha, v_prime, x_prime):
    mu = TateX(alpha,key,1)
    v = v_prime ^^ f_hash(mu)
    x = x_prime ^^ H(v)
    return (alpha, v, x)

alpha, v, x = in_decrypt(key,alpha,v_prime,x_prime)

print("in_decrypt(key,alpha,v_prime,x_prime):", (alpha, v, x))

in_decrypt(key,alpha,v_prime,x_prime): ((847233263721477088871220453940206233687146*z + 2595127466026680811584227734898508292904506 : 1147434789891369584664262580891559908408026*z + 2753689016240793237272970083846994781883175 : 1), 43, 120)



$$\mathsf{out\_ decrypt}(\alpha,v,x)\;\equiv$$
$$\vartheta\,a\gets H(v\oplus x)\,\centerdot\,\mathsf{if}\;\;\alpha\neq g(a)\;\;\mathsf{then}\;\;\texttt{fails}\;\;\mathsf{else}\;\;x$$

In [169]:
def out_decrypt(alpha, v, x):
    a = H(v ^^ x)
    if g(a) != alpha:
        return "Decryption failed"
    return x

print("out_decrypt(alpha, v, x):", out_decrypt(alpha, v, x))

out_decrypt(alpha, v, x): 120


$$\mathsf{Decrypt}(\mathit{key}, c) \;\equiv\;$$
$$\vartheta \,\alpha,v,x \gets \mathsf{in}(\mathit{key},c)\,\centerdot\,\mathsf{out}(\alpha,v,x)$$

In [170]:

def Decrypt(key, alpha, v_prime, x_prime):
    alpha, v, x = in_decrypt(key, alpha, v_prime, x_prime)
    return out_decrypt(alpha, v, x)

print("Decrypt(key, alpha, v_prime, x_prime):", Decrypt(key, alpha, v_prime, x_prime))

Decrypt(key, alpha, v_prime, x_prime): 120


In [171]:
lmbda = 128
id = "sender@test.com"
x = 7655465

s, beta = KeyGen(lmbda)
key = KeyExtract(id)
alpha, v_prime, x_prime = Encrypt(id, x)
decrypted_x = Decrypt(key, alpha, v_prime, x_prime)
print("Original message:", x)
print("Decrypted message:", decrypted_x)

Original message: 7655465
Decrypted message: 7655465
